In [1]:
import sys
import os
from pathlib import Path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('hey-database-tutorial')


api_key = os.getenv('OPENAI_API_KEY')
db_pwd = os.getenv('POSTGRES_PWD')

In [2]:
from src.config.config_loader import ConfigLoader

config = ConfigLoader.load_config(
    db_config_path=os.path.join(
        project_root, "configs", "DB_connections", "northwind_postgres.yaml"
    ),
    cache_config_path=os.path.join(
        project_root, "configs", "cache", "northwind_cache.yaml"
    ),
    sql_llm_config_path=os.path.join(
        project_root, "configs", "sql_llm", "openai_4o_mini.yaml"
    ),
    vector_store_config_path=os.path.join(
        project_root, "configs", "vector_store", "qdrant_northwind.yaml"
    ),
    prompt_config_path=os.path.join(project_root, "configs", "prompt.yaml"),
    metadata_config_path=os.path.join(project_root, "configs", "metadata_.yaml"),
    base_config_path=os.path.join(project_root, "configs", "base_config.yaml"),
)

In [3]:
from src.build.build_app_components import AppComponentsBuilder
app_components = AppComponentsBuilder(config).build()

/Users/giacomo/miniconda3/envs/heydatabase/lib/python3.10/site-packages/snowflake/sqlalchemy/base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)
INFO:hey-database:Connected to PostgreSQL database
INFO:hey-database:Vector store enabled, initializing client...


In [4]:
from src.metadata.metadata_startup import (
    MetadataManager,
    MetadataCacheHandler,
    MetadataProcessor,
)

metadata_processor = MetadataProcessor(
    table_extractor=app_components.table_metadata_extractor,
    column_extractor=app_components.column_metadata_extractor,
    table_enhancer=app_components.table_metadata_enhancer,
    column_enhancer=app_components.column_metadata_enhancer,
)
cache_handler = MetadataCacheHandler(app_components.cache)
metadata_manager = MetadataManager(metadata_processor, cache_handler)

In [5]:
metadata_manager.initialize_metadata()

ERROR:hey-database:Error reading metadata cache: [Errno 2] No such file or directory: 'data/cache/northwind/northwind_cache.json'
INFO:hey-database:No valid cache found, initializing metadata extraction and enrichment process.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
ERROR:hey-database:Error during metadata initialization: 'PostgresColumnMetadataExtractor' object has no attribute 'metadata_config'
Traceback (most recent call last):
  File "/Users/giacomo/IdeaProjects/hey-database/src/metadata/metadata_startup.py", line 148, in initialize_metadata
    self._current_state = self.metadata_processor.extract_and_enrich_metadata()
  File "/Users/giacomo/IdeaProjects/hey-database/src/metadata/metadata_startup.py", line 58, in extract_and_enrich_metadata
    base_columns_metadata = self.column_extractor.extract_metadata(table_name)
  File "/Users/giacomo/IdeaProjects/hey-database/src/metadata/extractors/column/column_metadata_extractor.py", lin

False